In [49]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import numpy as np
from numpy.random import seed
seed(1)
import requests
from tensorflow import random
random.set_seed(2)
import hvplot.pandas
%matplotlib inline

In [50]:
load_dotenv()

True

In [51]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
#print(alpaca_api_key)

In [52]:
#15Min interval
timeframe = "1D"
start_date = pd.Timestamp("2019-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()
#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["PYPL", "SQ", "AFRM", "MELI"]
fintech_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df
fintech_ticker.dropna(inplace=True)
fintech_ticker.head(5)
#apple_ticker.dropna(inplace=True)

AFRM                                        \
                              open     high     low    close      volume   
time                                                                       
2021-01-13 00:00:00-05:00   90.900  103.000   90.01   97.010  24984325.0   
2021-01-14 00:00:00-05:00  103.500  137.980  101.00  114.400  20358777.0   
2021-01-15 00:00:00-05:00  123.054  127.240  107.00  116.700   8033257.0   
2021-01-19 00:00:00-05:00  117.000  117.675  108.00  110.875   3116358.0   
2021-01-20 00:00:00-05:00  110.000  112.000  103.25  107.150   3504314.0   

                              MELI                                           \
                              open       high       low     close    volume   
time                                                                          
2021-01-13 00:00:00-05:00  1823.00  1830.0000  1766.685  1797.165  453625.0   
2021-01-14 00:00:00-05:00  1814.00  1889.6200  1814.000  1852.520  491020.0   
2021-01-15 00:00:00-05:00  1860.01  1877.1091  1805.460  1856.340  478130.0   
2021-01-19 00:00:00-05:00  1888.17  1938.1699  1861.000  1915.700  590186.0   
2021-01-20 00:00:00-05:00  1960.00  1991.8800  1929.520  1986.000  574762.0   

                             PYPL                                       \
                             open      high       low   close   volume   
time                                                                     
2021-01-13 00:00:00-05:00  236.57  247.4400  236.5400  244.90  6419870   
2021-01-14 00:00:00-05:00  246.87  249.8500  241.5400  242.01  5757075   
2021-01-15 00:00:00-05:00  242.25  244.5632  235.0341  239.79  6600442   
2021-01-19 00:00:00-05:00  243.30  248.7400  241.0000  247.24  5690793   
2021-01-20 00:00:00-05:00  248.85  249.6500  244.1400  244.19  5068278   

                               SQ                                      
                             open      high     low   close    volume  
time                                                                   
2021-01-13 00:00:00-05:00  228.95  234.8213  226.80  230.94  14553101  
2021-01-14 00:00:00-05:00  233.90  238.7600  229.59  232.71   6502923  
2021-01-15 00:00:00-05:00  233.63  236.4300  225.58  227.65   6344926  
2021-01-19 00:00:00-05:00  231.07  231.5900  223.25  227.25   6347656  
2021-01-20 00:00:00-05:00  230.12  230.8800  225.50  226.65   5379761

In [53]:
fintech_df = pd.DataFrame()
fintech_df["PayPal"] = fintech_ticker["PYPL"]["close"]
fintech_df["Square"] = fintech_ticker["SQ"]["close"]
fintech_df["Affirm"] = fintech_ticker["AFRM"]["close"]
fintech_df["Mercalibre"] = fintech_ticker["MELI"]["close"]
#fintech_df["Johnson&Johnson"] = nasdaq_ticker["JNJ"]["close"]
#apple_df = apple_df.pct_change().dropna()
fintech_df.index = fintech_df.index.date 
fintech_df.dropna(inplace=True)
fintech_df.head()

,PayPal,Square,Affirm,Mercalibre
2021-01-13,244.90,230.94,97.010,1797.165
2021-01-14,242.01,232.71,114.400,1852.520
2021-01-15,239.79,227.65,116.700,1856.340
2021-01-19,247.24,227.25,110.875,1915.700
2021-01-20,244.19,226.65,107.150,1986.000


In [54]:
#fintech_returns = fintech_df.pct_change().dropna()
#fintech_returns.head()

In [55]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [56]:
window_size = 10
feature_column = 1
target_column = 1
X, y = window_data(fintech_df, window_size, feature_column, target_column)
print (f"X sample values:\n{X[:5]} \n")
print (f"y sample values:\n{y[:5]}")

X sample values:
[[230.94 232.71 227.65 227.25 226.65 219.62 222.88 216.63 209.49 202.3 ]
 [232.71 227.65 227.25 226.65 219.62 222.88 216.63 209.49 202.3  219.91]
 [227.65 227.25 226.65 219.62 222.88 216.63 209.49 202.3  219.91 215.92]
 [227.25 226.65 219.62 222.88 216.63 209.49 202.3  219.91 215.92 221.94]
 [226.65 219.62 222.88 216.63 209.49 202.3  219.91 215.92 221.94 227.63]] 

y sample values:
[[219.91]
 [215.92]
 [221.94]
 [227.63]
 [227.76]]


In [57]:
print(X)

[[230.94 232.71 227.65 ... 216.63 209.49 202.3 ]
 [232.71 227.65 227.25 ... 209.49 202.3  219.91]
 [227.65 227.25 226.65 ... 202.3  219.91 215.92]
 ...
 [247.12 272.4  269.5  ... 272.93 271.01 268.57]
 [272.4  269.5  266.45 ... 271.01 268.57 267.94]
 [269.5  266.45 281.8  ... 268.57 267.94 263.48]]


In [58]:
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split :]
y_train = y[: split]
y_test = y[split :]

In [59]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [60]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.42480777]
  [0.44711963]
  [0.3596263 ]
  [0.35489593]
  [0.34780038]
  [0.26466414]
  [0.30321665]
  [0.22930464]
  [0.14486755]
  [0.05983917]]

 [[0.44711963]
  [0.38333543]
  [0.35489593]
  [0.34780038]
  [0.26466414]
  [0.30321665]
  [0.22930464]
  [0.14486755]
  [0.05983917]
  [0.26809366]]

 [[0.38333543]
  [0.37829321]
  [0.34780038]
  [0.26466414]
  [0.30321665]
  [0.22930464]
  [0.14486755]
  [0.05983917]
  [0.26809366]
  [0.22090823]]

 [[0.37829321]
  [0.37072986]
  [0.26466414]
  [0.30321665]
  [0.22930464]
  [0.14486755]
  [0.05983917]
  [0.26809366]
  [0.22090823]
  [0.29210028]]

 [[0.37072986]
  [0.28211269]
  [0.30321665]
  [0.22930464]
  [0.14486755]
  [0.05983917]
  [0.26809366]
  [0.22090823]
  [0.29210028]
  [0.35938978]]] 

X_test sample values:
[[[0.20736165]
  [0.24832976]
  [0.20001774]
  [0.15338221]
  [0.23509934]
  [0.26135289]
  [0.39865184]
  [0.36009934]
  [0.33006149]
  [0.46132923]]

 [[0.24832976]
  [0.21320434]
  [0.153382

In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [62]:
model = Sequential()

number_units = 5
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(units=number_units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [63]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [64]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 10, 5)             140       
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 5)             0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 10, 5)             220       
_________________________________________________________________
dropout_7 (Dropout)          (None, 10, 5)             0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 5)                 220       
_________________________________________________________________
dropout_8 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [65]:
model.fit(X_train, y_train, epochs=20, shuffle=False, batch_size=1, verbose=1)

Epoch 1/20
98/98 [==============================] - 4s 5ms/step - loss: 0.1609
Epoch 2/20
98/98 [==============================] - 0s 5ms/step - loss: 0.0826
Epoch 3/20
98/98 [==============================] - 0s 5ms/step - loss: 0.0745
Epoch 4/20
98/98 [==============================] - 0s 5ms/step - loss: 0.0751
Epoch 5/20
98/98 [==============================] - 0s 5ms/step - loss: 0.0714
Epoch 6/20
98/98 [==============================] - 0s 5ms/step - loss: 0.0754
Epoch 7/20
98/98 [==============================] - 0s 5ms/step - loss: 0.0738
Epoch 8/20
98/98 [==============================] - 1s 6ms/step - loss: 0.0644
Epoch 9/20
98/98 [==============================] - 1s 5ms/step - loss: 0.0677
Epoch 10/20
98/98 [==============================] - 1s 5ms/step - loss: 0.0687
Epoch 11/20
98/98 [==============================] - 1s 5ms/step - loss: 0.0677
Epoch 12/20
98/98 [==============================] - 0s 5ms/step - loss: 0.0668
Epoch 13/20
98/98 [==============================

In [66]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 1s 4ms/step - loss: 0.0763


0.07625136524438858

In [67]:
predicted = model.predict(X_test)

In [68]:
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [69]:
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
    }, index = fintech_df.index[-len(real_prices): ])
stocks.head(10)

,Real,Predicted
2021-06-18,237.17,225.158798
2021-06-21,233.91,225.829361
2021-06-22,235.98,226.449219
2021-06-23,238.69,227.049118
2021-06-24,244.28,227.636642
2021-06-25,240.24,228.221970
2021-06-28,246.60,228.728561
2021-06-29,248.88,229.199677
2021-06-30,243.74,229.770248
2021-07-01,241.96,230.258942


In [70]:
stocks.hvplot.line()#(label = "Predicting Apple's Close with the S&P500 Close - (6Day Window)", xlabel="Date", ylabel="Price")

:NdOverlay   [Variable]
   :Curve   [index]   (value)